In [9]:
!cat requirements.txt

aiohttp==3.12.13
boto3==1.38.39
botocore==1.38.39
sagemaker==2.247.0
litellm==1.72.2
strands-agents==0.1.6
strands-agents-builder==0.1.2
strands-agents-tools==0.1.4
matplotlib==3.10.3
pandas==2.3.0
seaborn==0.13.2
joblib==1.5.1
requests==2.32.4
uv==0.7.13

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
# Warnings are safe to ignore
%pip uninstall -q -y autogluon-multimodal autogluon-timeseries autogluon-features autogluon-common autogluon-core
%pip install -r requirements.txt -qU

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.38.39 which is incompatible.
sagemaker-studio-analytics-extension 0.2.0 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install -q boto3 python-docx PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [13]:
from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
!pip install --upgrade boto3

  Using cached boto3-1.40.10-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.40.10-py3-none-any.whl.metadata (5.7 kB)
Using cached boto3-1.40.10-py3-none-any.whl (140 kB)
Using cached botocore-1.40.10-py3-none-any.whl (14.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.38.39
    Uninstalling botocore-1.38.39:
      Successfully uninstalled botocore-1.38.39
  Attempting uninstall: boto3━━━━━━━━━━━━━━━━━━━ 0/2 [botocore]
    Found existing installation: boto3 1.38.390m 0/2 [botocore]
    Uninstalling boto3-1.38.39:━━━━━━━━━━━━━ 0/2 [botocore]
      Successfully uninstalled boto3-1.38.39 0/2 [botocore]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [boto3]32m1/2 [boto3]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.10 which is incompa

In [2]:
#from utils.strands_sagemaker import SageMakerAIModel
from strands.models.bedrock import BedrockModel

In [3]:
provider = "BEDROCK"  # Change this to SAGEMAKER to use the previously deployed endpoint instead of Bedrock

match provider:
    case "BEDROCK":
        # Using Claude 3.5 Sonnet from Bedrock
        model = BedrockModel(model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0")
    case "SAGEMAKER":
        # Using Qwen3 from our endpoint in SageMaker AI
        model = SageMakerAIModel({
            "endpoint_name": SAGEMAKER_ENDPOINT_NAME,
            "max_tokens": 16*1024,
            "temperature": 0.1,
            "stream": False
		})


In [4]:
# Import Required Libraries
import os
from strands_tools import http_request 
import json, time, uuid, re, requests
import io
import mimetypes
from typing import Tuple
try:
    import boto3
    from botocore.exceptions import BotoCoreError, ClientError
except Exception:
    boto3 = None

try:
    from PyPDF2 import PdfReader
except Exception:
    PdfReader = None

try:
    import docx  # python-docx
except Exception:
    docx = None


In [5]:
# ================== Analyzer (data-source agnostic, single entry) ==================
from strands import Agent, tool
import json, os, time, uuid, re, requests

# ---- environment time ----
os.environ["TZ"] = "America/New_York"
if hasattr(time, "tzset"):
    time.tzset()

# ---- config / paths ----
# Set this ONCE. It can be a URL (http/https) OR a local .txt file path.

#Servlet
'''
DATA_SOURCE   = "https://appserver2.sippasolutions.com/tomcat7/RemoteDBServlet-0.0.1-SNAPSHOT/DBServlet?secret=zaman@qcsurp2025clinical_use_case!"
ROW_DELIM     = "@"
DATA_LOG_FILE = "analyzer_raw_data_log.txt"
OUTPUT_JSONL  = "analyzer_outputs.jsonl"
'''

#txt file
'''
DATA_SOURCE   = "patient_summary.txt"
ROW_DELIM     = "@"
DATA_LOG_FILE = "analyzer_raw__patient_summary_data_log.txt"
OUTPUT_JSONL  = "analyzer_smart_goal_outputs.jsonl"
'''

#S3
DATA_SOURCE   = "s3://patient-summary-bucket/10_other.docx"
ROW_DELIM     = "@"
DATA_LOG_FILE = "analyzer_raw_s3_data_log.txt"
OUTPUT_JSONL  = "analyzer_s3_outputs.jsonl"



# ===== helpers =====
def _format_rows_as_lines(raw_text: str) -> str:
    """
    If the data uses '@' as a row delimiter, split onto newlines.
    Otherwise, return the text as-is (e.g., clinician notes).
    """
    text = raw_text.strip()
    if ROW_DELIM in text:
        chunks = [c.strip() for c in text.split(ROW_DELIM) if c.strip()]
        return "\n".join(chunks)
    return text

def _save_formatted_to_file(formatted_text: str, log_path: str):
    os.makedirs(os.path.dirname(log_path) or ".", exist_ok=True)
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(f"\n=== Run at {timestamp} ===\n")
        f.write(formatted_text + "\n")

def _coerce_json(text: str):
    """
    Extract the first JSON object from an LLM response and parse it.
    """
    s = str(text).strip()
    if s.startswith("{") and s.endswith("}"):
        return json.loads(s)
    m = re.search(r"\{.*\}", s, flags=re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in agent output.")
    return json.loads(m.group(0))

def _append_jsonl(path: str, obj: dict):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

# ===== S3 helpers =====
def _parse_s3_uri(uri: str) -> Tuple[str, str]:
    # s3://bucket/key -> (bucket, key)
    assert uri.lower().startswith("s3://"), "Not an s3:// URI"
    without = uri[5:]
    parts = without.split("/", 1)
    bucket = parts[0]
    key = parts[1] if len(parts) > 1 else ""
    return bucket, key

def _read_s3_object(uri: str) -> bytes:
    if boto3 is None:
        raise RuntimeError("boto3 not installed. Run: pip install boto3")
    bucket, key = _parse_s3_uri(uri)
    s3 = boto3.client("s3")  
    try:
        obj = s3.get_object(Bucket=bucket, Key=key)
        return obj["Body"].read()
    except (BotoCoreError, ClientError) as e:
        raise RuntimeError(f"S3 read failed for {uri}: {e}")

def _ext_or_mime(uri: str, content_bytes: bytes) -> str:
    # Guess from extension; if unknown, peek at header minimally
    mime, _ = mimetypes.guess_type(uri)
    return mime or "application/octet-stream"

def _extract_text_from_bytes(uri: str, content: bytes) -> str:
    mime = _ext_or_mime(uri, content)
    # Prefer extension-based routing
    luri = uri.lower()
    if luri.endswith(".pdf") or mime == "application/pdf":
        if PdfReader is None:
            raise RuntimeError("PyPDF2 not installed. Run: pip install PyPDF2")
        reader = PdfReader(io.BytesIO(content))
        parts = []
        for page in reader.pages:
            try:
                parts.append(page.extract_text() or "")
            except Exception:
                continue
        return "\n".join(p.strip() for p in parts if p)
    if luri.endswith(".docx") or mime in ("application/vnd.openxmlformats-officedocument.wordprocessingml.document",):
        if docx is None:
            raise RuntimeError("python-docx not installed. Run: pip install python-docx")
        d = docx.Document(io.BytesIO(content))
        return "\n".join(p.text for p in d.paragraphs if p.text)
    # Fallback: treat as UTF-8 text
    try:
        return content.decode("utf-8")
    except UnicodeDecodeError:
        return content.decode("latin-1", errors="ignore")

# ===== unified fetch tool (uses only DATA_SOURCE from config,  (URL / file / S3)) =====
@tool
def fetch_data() -> dict:
    """
    Fetches data from the configured DATA_SOURCE (URL or local file).
      - If DATA_SOURCE is http/https: HTTP POST with empty body {}
      - If DATA_SOURCE is a file path: read text from the file
      - If DATA_SOURCE is S3: s3://bucket/key: download securely using boto3 and extract text (PDF/DOCX/TXT)

    Returns:
      {
        "raw_text": string,
        "formatted_text": string,   # '@' rows split to newlines if present
        "meta": {"source_type": "url"|"file"|"s3", "data_source": string}
      }
    """
    ds = DATA_SOURCE


    # S3 branch
    if isinstance(ds, str) and ds.lower().startswith("s3://"):
        try:
            blob = _read_s3_object(ds)
            raw_text = _extract_text_from_bytes(ds, blob)
        except Exception as e:
            return {"error": f"S3 error: {e}", "raw_text": "", "formatted_text": "", "meta": {"source_type": "s3", "data_source": ds}}
        formatted = _format_rows_as_lines(raw_text)
        # Optional audit log:
        _save_formatted_to_file(formatted, DATA_LOG_FILE)
        return {"raw_text": raw_text, "formatted_text": formatted, "meta": {"source_type": "s3", "data_source": ds}}
    
    # URL branch
    if isinstance(ds, str) and ds.lower().startswith(("http://", "https://")):
        try:
            resp = requests.post(ds, data={}, timeout=60)
            resp.raise_for_status()
            raw = resp.text
        except Exception as e:
            return {"error": f"HTTP error: {e}", "raw_text": "", "formatted_text": "", "meta": {"source_type": "url", "data_source": ds}}
        formatted = _format_rows_as_lines(raw)
        # Log servlet-like pulls for audit
        _save_formatted_to_file(formatted, DATA_LOG_FILE)
        return {"raw_text": raw, "formatted_text": formatted, "meta": {"source_type": "url", "data_source": ds}}

    # File branch
    if isinstance(ds, str) and os.path.exists(ds):
        try:
            # If it's a binary (pdf/docx), read bytes and extract text
            if ds.lower().endswith((".pdf", ".docx")):
                with open(ds, "rb") as f:
                    content = f.read()
                raw_text = _extract_text_from_bytes(ds, content)
            else:
                with open(ds, "r", encoding="utf-8") as f:
                    raw_text = f.read()
        except Exception as e:
            return {"error": f"File read error: {e}", "raw_text": "", "formatted_text": "", "meta": {"source_type": "file", "data_source": ds}}
        formatted = _format_rows_as_lines(raw_text)
        return {"raw_text": raw_text, "formatted_text": formatted, "meta": {"source_type": "file", "data_source": ds}}

    # Unknown source
    return {"error": f"DATA_SOURCE not found or unsupported: {ds}", "raw_text": "", "formatted_text": "", "meta": {"source_type": "unknown", "data_source": str(ds)}}

# ===== generic analyzer prompt =====
# Edit ONLY this system prompt to change the task and required output JSON.
ANALYZER_PROMPT = """
You are an Analyzer Agent.

Tool available:
- fetch_data() -> {raw_text, formatted_text, meta}

You will receive an empty JSON input. IGNORE any user content and:
INSTRUCTIONS:
1) Call fetch_data EXACTLY ONCE (no arguments). It reads the configured DATA_SOURCE.
2) Use "formatted_text" as your working input. It is newline-separated if the source used '@' row delimiters; otherwise it may be free text/paragraphs.
3) Perform the analysis according to the TASK below.
4) Produce output that matches the OUTPUT CONTRACT below EXACTLY (keys and structure). Output ONLY that JSON object and nothing else.
5) Do not call any other tools. Do not print anything except the final JSON. Do not retry fetch_data.

DEFAULT TASK (you may replace this entire task text to repurpose the agent):
- Derive from the following content SMART goals that are specific, measurable, actionable, relevant and time-bounded: {DATA_SOURCE}"


DEFAULT OUTPUT CONTRACT (you may replace this to another JSON contract):
{
  "smart_goals": [
    {
      "goal_number": "integer (starts at 1 and increments for each goal)",
      "description": "string (time-bound, measurable details)"
    }
  ]
}
"""

analyzer_agent = Agent(
    model=model,
    system_prompt=ANALYZER_PROMPT,
    tools=[fetch_data],
)

# ===== Run the agent directly =====
payload = {}  # always empty
raw_response = analyzer_agent(json.dumps(payload))

# Parse to clean JSON and save
parsed = _coerce_json(raw_response)
_append_jsonl(OUTPUT_JSONL, {
    "run_id": str(uuid.uuid4()),
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
    "data_source": DATA_SOURCE,
    "analyzer_output": parsed
})




Tool #1: fetch_data
{
  "smart_goals": [
    {
      "goal_number": 1,
      "description": "Reduce HbA1c from current 8.1% to below 7.0% within 3 months through medication adherence, blood glucose monitoring 4 times daily, and lifestyle modifications"
    },
    {
      "goal_number": 2,
      "description": "Achieve and maintain fasting blood glucose between 80-130 mg/dL and post-meal readings under 180 mg/dL within 4 weeks through proper medication timing and balanced meal planning"
    },
    {
      "goal_number": 3,
      "description": "Complete 150 minutes of moderate-intensity aerobic exercise (such as brisk walking or swimming) per week, plus strength training twice weekly, starting immediately"
    },
    {
      "goal_number": 4,
      "description": "Reduce weight from 185 lbs to 160-165 lbs over the next 6 months through portion control and balanced nutrition plan including 45-60 grams of carbs per meal"
    },
    {
      "goal_number": 5,
      "description": "Lower LD

In [6]:
from strands import Agent, tool
import json, os, time, uuid, re
from statistics import mean

# ---------- Paths ----------
ANALYZER_JSONL = "analyzer_smart_goal_outputs.jsonl"     # produced by the Analyzer
CLINICIAN_JSON = "clinician_evaluation.json"  # optional: only used for engagement eval
EVAL_JSONL     = "evaluator_runs.jsonl"

# ---------- File helpers ----------
def _read_jsonl(path: str):
    items = []
    if not os.path.exists(path):
        return items
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                items.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return items

def _read_json(path: str):
    if not os.path.exists(path):
        return []
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def _append_jsonl(path: str, obj: dict):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

def _coerce_json(text: str):
    s = str(text).strip()
    if s.startswith("{") and s.endswith("}"):
        return json.loads(s)
    m = re.search(r"\{.*\}", s, flags=re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in evaluator output.")
    return json.loads(m.group(0))

# ---------- Low-level loaders as tools ----------
@tool
def load_analyzer_runs(limit: int | None = None) -> dict:
    """
    Load analyzer outputs (JSONL), sorted by timestamp ASC. Optionally keep only latest 'limit'.
    """
    runs = _read_jsonl(ANALYZER_JSONL)
    runs.sort(key=lambda r: r.get("timestamp", ""))
    if limit:
        runs = runs[-limit:]
    return {"runs": runs}

@tool
def load_clinician_eval() -> dict:
    """
    Load clinician_evaluation.json (array), sorted by timestamp ASC (optional for SMART).
    """
    items = _read_json(CLINICIAN_JSON)
    items.sort(key=lambda r: r.get("timestamp", ""))
    return {"items": items}

# ---------- Build Engagement pairs: analyzer vs clinician ----------
def _build_engagement_cases(runs, clinician_items):
    # index clinician by timestamp -> {device_id -> rec}
    clin_index = {}
    for row in clinician_items:
        ts = row.get("timestamp", "")
        for rec in row.get("clinician_output", {}).get("recommendations", []):
            clin_index.setdefault(ts, {})[rec.get("device_id")] = {
                "category_recommended": rec.get("category_recommended"),
                "rationale": rec.get("rationale"),
            }

    cases = []
    for run in runs:
        ts = run.get("timestamp", "")
        recs = (run.get("analyzer_output") or {}).get("recommendations", [])
        for rec in recs:
            dev = rec.get("device_id")
            analyzer_rec = {
                "category_recommended": rec.get("category_recommended"),
                "rationale": rec.get("rationale"),
            }
            clinician_rec = (clin_index.get(ts, {}) or {}).get(dev)
            if clinician_rec:
                cases.append({
                    "case_id": f"{ts}::{dev}",
                    "timestamp": ts,
                    "device_id": dev,
                    "analyzer": analyzer_rec,
                    "clinician": clinician_rec,
                })
    return cases

# ---------- Build SMART-goal cases (rubric-driven) ----------
def _build_smart_goal_cases(runs):
    """
    Flatten SMART goals from analyzer_output.smart_goals.
    Each case contains goal text plus a default SMART rubric the judge can use.
    """
    cases = []
    for run in runs:
        ts = run.get("timestamp", "")
        ao = run.get("analyzer_output") or {}
        goals = ao.get("smart_goals") or []
        for g in goals:
            num = g.get("goal_number")
            desc = g.get("description", "")
            cases.append({
                "case_id": f"{ts}::goal_{num}",
                "timestamp": ts,
                "goal_number": num,
                "goal_text": desc,
            })
    return cases

# ---------- Planning tool that abstracts use cases ----------
@tool
def build_eval_plan(limit: int | None = None) -> dict:
    """
    Decide which evaluation to run based on analyzer_outputs.jsonl contents.
    Returns a plan with:
      {
        "evaluation_type": "engagement_vs_clinician" | "smart_goals_rubric",
        "metrics": ["..."],
        "rubric": { ... optional ... },
        "cases": [ ... normalized cases ... ]
      }
    """
    runs = load_analyzer_runs(limit=limit)["runs"]
    clinicians = load_clinician_eval()["items"]

    # Heuristic: if analyzer_output has 'recommendations' (device_id...), prefer engagement;
    # if analyzer_output has 'smart_goals', prefer SMART rubric mode.
    has_engagement = any((r.get("analyzer_output") or {}).get("recommendations") for r in runs)
    has_smart = any((r.get("analyzer_output") or {}).get("smart_goals") for r in runs)

    if has_engagement and clinicians:
        cases = _build_engagement_cases(runs, clinicians)
        return {
            "evaluation_type": "engagement_vs_clinician",
            "metrics": ["correctness", "completeness", "helpfulness", "coherence", "relevance"],
            "rubric": {
                "notes": "Compare analyzer category & rationale to clinician's.",
                "agreement_rules": {
                    "match": "same category (case-insensitive)",
                    "partial": "different category but rationale overlaps clinician intent",
                    "mismatch": "different with little/no overlap",
                }
            },
            "cases": cases
        }

    if has_smart:
        cases = _build_smart_goal_cases(runs)
        return {
            "evaluation_type": "smart_goals_rubric",
            "metrics": ["specific", "measurable", "achievable", "relevant", "time_bound", "clarity"],
            "rubric": {
                "specific":   "Clearly states the behavior/target (who/what/when/where).",
                "measurable": "Includes a quantifiable criterion (count, frequency, value).",
                "achievable": "Feasible for the patient (resources/constraints).",
                "relevant":   "Aligned to diabetes/health needs in the notes.",
                "time_bound": "Contains a concrete timeframe or deadline.",
                "clarity":    "Readable, unambiguous, free of contradictions."
            },
            "cases": cases
        }

    # Fallback: nothing to evaluate
    return {
        "evaluation_type": "none",
        "metrics": [],
        "rubric": {},
        "cases": []
    }

# ---------- Evaluator Agent (general, plan-driven) ----------
EVALUATOR_PROMPT = """
You are an Evaluator (LLM-as-Judge) that supports multiple evaluation modes via a plan.

You will be given a plan from the tool build_eval_plan(limit) with:
- evaluation_type: "engagement_vs_clinician" or "smart_goals_rubric"
- metrics: list of metric names to score in [0.0, 1.0]
- rubric: guidance for scoring
- cases: a list of cases to evaluate

CALLS:
1) Call build_eval_plan(limit) EXACTLY ONCE (use the user-provided {"limit":N} if present; otherwise none).

SCORING:
- For "engagement_vs_clinician":
  Each case has:
    { case_id, timestamp, device_id, analyzer{category_recommended, rationale}, clinician{category_recommended, rationale} }
  Score metrics: correctness, completeness, helpfulness, coherence, relevance.
  Also produce:
    agreement = "match" | "partial" | "mismatch"
  Rules:
    - match if categories are the same (case-insensitive).
    - partial if different but analyzer rationale substantially overlaps clinician intent.
    - mismatch otherwise.

- For "smart_goals_rubric":
  Each case has:
    { case_id, timestamp, goal_number, goal_text }
  Score metrics: specific, measurable, achievable, relevant, time_bound, clarity.
  Focus only on the goal_text vs rubric. If unsafe, note it briefly.

OUTPUT: STRICT JSON ONLY:
{
  "evaluation_type": "string",
  "cases_scored": 0,
  "scores": [
    {
      "case_id": "string",
      "metric_scores": { "<metric>": 0.0 },
      "agreement": "match|partial|mismatch|n/a",
      "notes": "short justification (<=40 words)"
    }
  ]
}

PROCESS:
- Produce one score object per case with values in [0.0, 1.0].
- Use "agreement":"n/a" for smart_goals_rubric (no clinician).
- Keep notes concise and specific.
"""

evaluator_agent = Agent(
    model=model,
    system_prompt=EVALUATOR_PROMPT,
    tools=[build_eval_plan],  # single entry tool that returns everything needed
)

# ---------- Runner (single call) ----------
def run_evaluator(limit: int | None = None, print_json: bool = True):
    payload = {"limit": limit} if limit else {}
    raw = evaluator_agent(json.dumps(payload))
    out = _coerce_json(raw)

    # Compute overall means dynamically across whatever metric set came back
    metrics = set()
    for s in out.get("scores", []):
        for k in (s.get("metric_scores") or {}).keys():
            metrics.add(k)
    metrics = sorted(metrics)

    buckets = {k: [] for k in metrics}
    for s in out.get("scores", []):
        for k in metrics:
            v = (s.get("metric_scores") or {}).get(k)
            if isinstance(v, (int, float)):
                buckets[k].append(float(v))

    overall = {k: (round(mean(buckets[k]), 4) if buckets[k] else 0.0) for k in metrics}
    out["cases_scored"] = len(out.get("scores", []))
    out["overall"] = overall

    _append_jsonl(EVAL_JSONL, {
        "run_id": str(uuid.uuid4()),
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()),
        "evaluator_output": out
    })

    return out

# ---- run ----
run_evaluator()           # evaluate all available cases


I'll help evaluate the cases. Let me first get the evaluation plan.
Tool #2: build_eval_plan
I'll evaluate these SMART goals using the provided rubric. Here's my evaluation in the required JSON format:

{
  "evaluation_type": "smart_goals_rubric",
  "cases_scored": 10,
  "scores": [
    {
      "case_id": "2025-08-14 00:56:21::goal_1",
      "metric_scores": {
        "specific": 1.0,
        "measurable": 1.0,
        "achievable": 0.9,
        "relevant": 1.0,
        "time_bound": 1.0,
        "clarity": 1.0
      },
      "agreement": "n/a",
      "notes": "Clear target values, monitoring frequency, and timeframe. Highly specific and measurable glucose management plan."
    },
    {
      "case_id": "2025-08-14 00:56:21::goal_2",
      "metric_scores": {
        "specific": 1.0,
        "measurable": 1.0,
        "achievable": 0.9,
        "relevant": 1.0,
        "time_bound": 0.7,
        "clarity": 1.0
      },
      "agreement": "n/a",
      "notes": "Precise exercise requireme

{'evaluation_type': 'smart_goals_rubric',
 'cases_scored': 10,
 'scores': [{'case_id': '2025-08-14 00:56:21::goal_1',
   'metric_scores': {'specific': 1.0,
    'measurable': 1.0,
    'achievable': 0.9,
    'relevant': 1.0,
    'time_bound': 1.0,
    'clarity': 1.0},
   'agreement': 'n/a',
   'notes': 'Clear target values, monitoring frequency, and timeframe. Highly specific and measurable glucose management plan.'},
  {'case_id': '2025-08-14 00:56:21::goal_2',
   'metric_scores': {'specific': 1.0,
    'measurable': 1.0,
    'achievable': 0.9,
    'relevant': 1.0,
    'time_bound': 0.7,
    'clarity': 1.0},
   'agreement': 'n/a',
   'notes': 'Precise exercise requirements but lacks overall timeframe for goal completion.'},
  {'case_id': '2025-08-14 00:56:21::goal_3',
   'metric_scores': {'specific': 0.9,
    'measurable': 1.0,
    'achievable': 0.8,
    'relevant': 1.0,
    'time_bound': 1.0,
    'clarity': 0.9},
   'agreement': 'n/a',
   'notes': 'Clear target weight and timeline, but 